<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse LDA
    - Entrainement LDA
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 18/07/2020 

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')

In [ ]:
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 

In [ ]:
tagText = TagText()
print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)

tagText.nombre_post_entree = 50000
tagText.precision = 50000

data_question = tagText.read_source()
data_question.head()


In [ ]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed[1:9])

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Train/Test</p>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Sampling dataset
vectorizer_X = TfidfVectorizer(
            max_df=0.95,
            min_df=2,
            max_features=50000,
            stop_words='english'
                              )

#Y
y_all = [
    item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
    for item in data_question['SelectedTags']
]

#print(y_train_tag)
lb = tagText.MultiLabelBinarizer()
Y_all = lb.fit_transform(y_all)


# 80/20 split
X_lda_train, X_lda_test, y_lda_train, y_lda_test = train_test_split(
    data_preprocessed, y_all, test_size=0.2,train_size=0.8, random_state=0)
y_lda_train = lb.transform(y_lda_train)
y_lda_test = lb.transform(y_lda_test)

# TF-IDF matrices
X_tfidf_train = vectorizer_X.fit_transform(X_lda_train)
X_tfidf_test = vectorizer_X.transform(X_lda_test)

In [ ]:
print(y_lda_test)
print(Y_all)

In [ ]:
print(len(y_lda_train))

In [ ]:
type(X_tfidf_train)
print(X_tfidf_train)

In [ ]:
idf = vectorizer_X.idf_
print(idf)

In [ ]:
dd=dict(zip(vectorizer_X.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])

In [ ]:
'''
print(dd['like'])
print(dd['consolewritelinesecond'])  # like is most common and forecast is least common among the news headlines.
'''

In [ ]:
print(dd)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(X_tfidf_train)


In [ ]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

In [ ]:
type(lda_top)
print(lda_top)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(
        n_estimators=100,
        max_depth=2,
        random_state=0
)
#lda_top=lda_model.fit_transform(X_tfidf_train)
clf.fit(lda_top, y_lda_train)

In [ ]:
classes = lb.classes_

no_top_words = 5
text = "git is the best source code"
text = tagText.preprocessing(text)
text = [text]
mytext = vectorizer_X.transform(text)
text_projection = lda_model.transform(mytext)
'''
print (text_projection)
print (text_projection.shape)
'''
predicted = clf.predict_proba(text_projection)
'''
print (predicted)
print (len(predicted))
'''
tempTag = [(1-item[0][0]) for item in predicted]
list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
'''
tempTag = predicted[0]
list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
'''
'''
print (predicted)
print (predicted.shape)
'''
list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
#print (list_id_sorted)

list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
#print(list_id_sorted_suggested)

print (str([classes[id] for id in list_id_sorted_suggested]))

In [ ]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 5
    for i in range(x_test.shape[0]):
        #for i in range(x_test.shape[0]):
        text_projection = x_test
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        predicted = clf.predict_proba(text_projection)
        '''
        print (predicted)
        print (len(predicted))
        '''
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        #print (str(prediction))
        
        
        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)
        
        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """

        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        #str([tag for tag in y_true[i]]if tag ==1)
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))

scoring(X_tfidf_test, y_lda_test, clf, lb, True, lda_model)

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-a - Data - Tag generator - Unsupervised - LDA</p>

In [ ]:
def lda_prepare_tag(tagText, data_preprocessed, data_tag, no_tropics=32):
        '''
        prepare lda, topic ad tf vectorizer from data preprocessed
        '''
        from sklearn.ensemble import RandomForestClassifier
        
        tagText.n_topic = no_tropics
        documents = data_preprocessed[0:tagText.precision]#.unique()
        lda_tf, lda_tf_vectorizer = tagText.lda_init(documents)
        
        lda, score, perplexity = tagText.lda_train(lda_tf, no_tropics)
        
        print("Log Likelihood: ", score)
        print("Perplexity: ", perplexity)

        # See model parameters
        print(lda.get_params())

        lda_topicnames = ["Topic" + str(i) for i in range(lda.n_topics)]
        # Topic-Keyword Matrix
        lda_df_topic_keywords = tagText.pd.DataFrame(lda.components_)
        
        # Assign Column and Index
        lda_df_topic_keywords.columns = lda_tf_vectorizer.get_feature_names()
        lda_df_topic_keywords.index = lda_topicnames
        
        #tag
        topic_word = lda.topic_word_
        print("type(topic_word): {}".format(type(topic_word)))
        print("shape: {}".format(topic_word.shape))
        
        clf = RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
        )
        clf.fit(lda, y_lda_train)
        
        return lda, lda_df_topic_keywords, lda_tf_vectorizer, clf

In [ ]:
lda_tagText, df_topic_keywords, tf_vectorizer, clf = lda_prepare_tag(tagText, X_tfidf_train, y_lda_train, 32)


In [ ]:
def average_jaccard(y_true,y_pred):

    ''' It calculates Jaccard similarity coefficient score for each instance,and
    it finds their averange in percentage
    Parameters:
    y_true: truth labels
    y_pred: predicted labels
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

In [ ]:
# most important words for each topic
vocab = vectorizer_X.get_feature_names()

for i, comp in enumerate(lda_tagText.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")